In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/src')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/notebooks')

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import datetime
from utils import *
# from _utils import filter_df_on_conditions, make_subplots
import seaborn as sns
from functools import partial

In [4]:
datetime.datetime.now(datetime.timezone.utc).isoformat()

'2024-02-29T23:14:43.183446+00:00'

In [5]:
# wandb.init(project="fwal",entity="evangeorgerex")
api = wandb.Api(timeout=60) #, entity='evangeorgerex')

In [6]:
entity = "evangeorgerex"
project_name = "fwal"
sweep_id = "scy32mac"

# Fetch the sweep
api = wandb.Api()
sweep = api.sweep(f"{entity}/{project_name}/{sweep_id}")


In [7]:
run = sweep.runs[0]

In [8]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: evangeorgerex (l65). Use `wandb login --relogin` to force relogin


True

In [9]:
history_data = []

# Iterate over the scan_history generator
for metrics in run.scan_history():
    history_data.append(metrics)

# Convert the list of dictionaries into a DataFrame
df1 = pd.DataFrame(history_data)
    
history_data = []
# Iterate over the scan_history generator
for metrics in run.scan_history():
    history_data.append(metrics)

# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(history_data)

df1.equals(df2)

True

In [10]:

# Data structure to hold all sweep data
sweep_data = []

# Iterate over each run in the sweep
for run in sweep.runs:
    # Fetching run summary and config data
    run_summary = run.summary._json_dict
    run_config = {k: v for k, v in run.config.items() if not k.startswith('_')}
    
    history_data = []
    
    # Iterate over the scan_history generator
    for metrics in run.scan_history():
        history_data.append(metrics)

    # Convert the list of dictionaries into a DataFrame
    history_df = pd.DataFrame(history_data)

   
    # Store run data in a dictionary and append to the list
    run_data = {
        "id": run.id,
        "summary": run_summary,
        "config": run_config,
        "history": history_df.to_dict() if 'history_df' in locals() else None  # Uncomment if history data is added
    }
    sweep_data.append(run_data)



In [11]:
data = []

# Assuming sweep.runs is a list of runs you want to process

for run in sweep_data:
    run_id = run['id']
    run_summary = run['summary']
    run_config = run['config']
    history_df = pd.DataFrame.from_dict(run['history'])
    
    
    history_df['dataset'] = run_config['dataset']
    history_df['id'] = run_id
    
    history_df['bestmodel_valid/balanced_accuracy'] = run_summary['bestmodel_valid/balanced_accuracy']
    history_df['bestmodel_valid/mask'] = run_summary['best_mask'] 
    
    rounded_str_list = [f"{round(num, 2):.2f}" for num in run_summary['best_mask_parameters']]

    # Convert the list of strings to a single string representation of the list
    final_str = "[" + ", ".join(rounded_str_list) + "]"

    history_df['bestmodel_valid/mask_parameters'] =  final_str
    
    # Append the augmented DataFrame to our list
    data.append(history_df)

# If you have multiple runs and want to combine them, you can do so. Here's how to concatenate all the DataFrames:
combined_df = pd.concat(data, ignore_index=True)


In [15]:
combined_df['max_balanced_accuracy'] = combined_df.groupby('id')['masks/valid_metrics.balanced_accuracy'].transform(max)

# Step 2: Filter to keep only rows where 'masks/valid_metrics.balanced_accuracy' matches the max for that 'id'
max_accuracy_rows = combined_df[combined_df['masks/valid_metrics.balanced_accuracy'] == combined_df['max_balanced_accuracy']]

# Drop the auxiliary column 'max_balanced_accuracy' if it's no longer needed
max_accuracy_rows = max_accuracy_rows.drop(columns=['max_balanced_accuracy'])

max_accuracy_rows.groupby(['dataset','id','masks/mask']).agg({
    # 'id': 'first',
    'masks/valid_metrics.balanced_accuracy': 'mean',
    'bestmodel_valid/balanced_accuracy': 'mean',
    # 'bestmodel/step': 'mean',
    'bestmodel_valid/mask': 'first',
    'bestmodel_valid/mask_parameters':'first'
    }).dropna()

masks/valid_metrics.balanced_accuracy  \
dataset                          id       masks/mask                                           
simple_trig_synth                2kr38xi8 11010                                     0.982000   
                                 avsglt2n 10001                                     0.958500   
                                 cnopwvct 11010                                     0.978500   
summed_squares_exponential_synth m8nausgt 10000011000                               0.686578   
                                 qjq3rpu0 00000000001                               0.687095   
                                 ryd2f0xe 00000001000                               0.679682   
trigonometric_polynomial_synth   8678foa4 00000001000                               0.995158   
                                 b6dvuub5 00000011000                               0.996177   
                                 wubz64gh 00001010001                               0.996942   

                                                       bestmodel_valid/balanced_accuracy  \
dataset                          id       masks/mask                                       
simple_trig_synth                2kr38xi8 11010                                 0.976138   
                                 avsglt2n 10001                                 0.939388   
                                 cnopwvct 11010                                 0.978959   
summed_squares_exponential_synth m8nausgt 10000011000                           0.664913   
                                 qjq3rpu0 00000000001                           0.669124   
                                 ryd2f0xe 00000001000                           0.665896   
trigonometric_polynomial_synth   8678foa4 00000001000                           0.997545   
                                 b6dvuub5 00000011000                           0.973048   
                                 wubz64gh 00001010001                           0.985529   

                                                      bestmodel_valid/mask  \
dataset                          id       masks/mask                         
simple_trig_synth                2kr38xi8 11010                      11010   
                                 avsglt2n 10001                      10001   
                                 cnopwvct 11010                      11010   
summed_squares_exponential_synth m8nausgt 10000011000          10010011000   
                                 qjq3rpu0 00000000001          10001011001   
                                 ryd2f0xe 00000001000          11110001000   
trigonometric_polynomial_synth   8678foa4 00000001000          00000001000   
                                 b6dvuub5 00000011000          10000011000   
                                 wubz64gh 00001010001          00001000001   

                                                                         bestmodel_valid/mask_parameters  
dataset                          id       masks/mask                                                      
simple_trig_synth                2kr38xi8 11010                         [0.01, 0.14, -0.16, 0.59, -0.74]  
                                 avsglt2n 10001                        [0.04, -0.18, -0.43, -1.27, 0.97]  
                                 cnopwvct 11010                         [0.95, 0.00, -2.72, 0.69, -1.02]  
summed_squares_exponential_synth m8nausgt 10000011000  [0.81, -0.62, -2.30, 0.00, -1.26, -1.87, 0.10,...  
                                 qjq3rpu0 00000000001  [0.01, -0.30, -0.56, -1.30, 0.59, -1.12, 0.09,...  
                                 ryd2f0xe 00000001000  [0.51, 0.22, 0.02, 0.52, -0.41, -0.20, -1.60, ...  
trigonometric_polynomial_synth   8678foa4 00000001000  [-1.11, -1.28, -1.37, -1.19, -1.38, -0.25, -1....  
                                 b6dvuub5 00000011000  [0.01, -1.16, -2.68, -0.66, -1.40, -1.91, 0.27...  
                                 wubz64gh 00001010001  [

In [14]:
combined_df['max_balanced_accuracy'] = combined_df.groupby('id')['masks/valid_metrics.balanced_accuracy'].transform(max)

# Step 2: Filter to keep only rows where 'masks/valid_metrics.balanced_accuracy' matches the max for that 'id'


masks/test_metrics.balanced_accuracy  \
dataset                          id       masks/mask                                          
simple_trig_synth                2kr38xi8 11010                                    0.978500   
                                 avsglt2n 10001                                    0.953500   
                                 cnopwvct 11010                                    0.981000   
summed_squares_exponential_synth m8nausgt 10000011000                              0.674309   
                                 qjq3rpu0 00000000001                              0.675325   
                                 ryd2f0xe 00000001000                              0.666557   
trigonometric_polynomial_synth   8678foa4 00000001000                              0.992100   
                                 b6dvuub5 00000011000                              0.995923   
                                 wubz64gh 00001010001                              0.969352   

                                                       masks/valid_metrics.balanced_accuracy  \
dataset                          id       masks/mask                                           
simple_trig_synth                2kr38xi8 11010                                     0.982000   
                                 avsglt2n 10001                                     0.958500   
                                 cnopwvct 11010                                     0.978500   
summed_squares_exponential_synth m8nausgt 10000011000                               0.686578   
                                 qjq3rpu0 00000000001                               0.687095   
                                 ryd2f0xe 00000001000                               0.679682   
trigonometric_polynomial_synth   8678foa4 00000001000                               0.995158   
                                 b6dvuub5 00000011000                               0.996177   
                                 wubz64gh 00001010001                               0.996942   

                                                       bestmodel_valid/balanced_accuracy  \
dataset                          id       masks/mask                                       
simple_trig_synth                2kr38xi8 11010                                 0.976138   
                                 avsglt2n 10001                                 0.939388   
                                 cnopwvct 11010                                 0.978959   
summed_squares_exponential_synth m8nausgt 10000011000                           0.664913   
                                 qjq3rpu0 00000000001                           0.669124   
                                 ryd2f0xe 00000001000                           0.665896   
trigonometric_polynomial_synth   8678foa4 00000001000                           0.997545   
                                 b6dvuub5 00000011000                           0.973048   
                                 wubz64gh 00001010001                           0.985529   

                                                      bestmodel_valid/mask  \
dataset                          id       masks/mask                         
simple_trig_synth                2kr38xi8 11010                      11010   
                                 avsglt2n 10001                      10001   
                                 cnopwvct 11010                      11010   
summed_squares_exponential_synth m8nausgt 10000011000          10010011000   
                                 qjq3rpu0 00000000001          10001011001   
                                 ryd2f0xe 00000001000          11110001000   
trigonometric_polynomial_synth   8678foa4 00000001000          00000001000   
                                 b6dvuub5 00000011000          10000011000   
                                 wubz64gh 00001010001          00001000001   

                                                                         bestmod

In [ ]:
max_accuracy_rows = combined_df[combined_df['masks/valid_metrics.balanced_accuracy'] == combined_df['max_balanced_accuracy']]

# Drop the auxiliary column 'max_balanced_accuracy' if it's no longer needed
max_accuracy_rows = max_accuracy_rows.drop(columns=['max_balanced_accuracy'])

max_accuracy_rows.groupby(['dataset','id','masks/mask']).agg({
    # 'id': 'first',
    'masks/test_metrics.balanced_accuracy': 'mean',
    'masks/valid_metrics.balanced_accuracy': 'mean',
    'bestmodel_valid/balanced_accuracy': 'mean',
    'bestmodel_valid/mask': 'first',
    'bestmodel_valid/mask_parameters':'first'
    }).dropna()